In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("lab-ml").getOrCreate()
spark

In [3]:

df = spark.read.parquet("s3://aws-logs-780494741284-us-east-1/dataset/project_graph_data")

In [4]:
df.createOrReplaceTempView("df")
df.printSchema()

root
 |-- source: string (nullable = true)
 |-- target: string (nullable = true)



In [5]:
df.take(5)

[Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.HB6611.COM/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.SX1822.COM/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.DK8883.COM/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.287720.COM/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.MG8541.COM/')]

# Combine all urls together

In [9]:
df1 = df.select(['target'])
df2 = df.select(['source'])



In [10]:
df1 = df1.selectExpr("target as urls")
df1.show()
df1.printSchema()

+-------------------+
|               urls|
+-------------------+
|   /WWW.HB6611.COM/|
|   /WWW.SX1822.COM/|
|   /WWW.DK8883.COM/|
|   /WWW.287720.COM/|
|   /WWW.MG8541.COM/|
|  /WWW.1382121.COM/|
|  /WWW.3615005.COM/|
|     /WWW.7760.NET/|
| /WWW.HG123777.COM/|
|  /WWW.CP85555.COM/|
|    /WWW.99662.COM/|
| /WWW.13300007.COM/|
|/WWW.XPJ005005.COM/|
|    /WWW.GWSSC.COM/|
|  /WWW.BMW6609.COM/|
|  /WWW.WXC7779.COM/|
|    /WWW.CR949.COM/|
|  /WWW.8899480.COM/|
|   /WWW.142VNS.COM/|
|  /WWW.DGD0000.COM/|
+-------------------+
only showing top 20 rows

root
 |-- urls: string (nullable = true)



In [11]:
df2 = df2.selectExpr("source as urls")
df2.show()
df2.printSchema()

+--------------------+
|                urls|
+--------------------+
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
+--------------------+
only showing top 20 rows

root
 |-- urls: string (nullable = true)



In [34]:
df_c = df1.unionByName(df2)
df_c.printSchema()

root
 |-- urls: string (nullable = true)



In [37]:
from operator import itemgetter

indexer = (df_c.select("urls").distinct()
    .rdd.map(itemgetter(0)).zipWithIndex()
    .toDF(["urls", "label"]))

df_c = df_c.join(indexer, ["urls"])

In [38]:
df_c.show(3)

+-----------------+-----+
|             urls|label|
+-----------------+-----+
| /WWW.MG3446.COM/|    0|
| /WWW.MG3446.COM/|    0|
|/WWW.BET5571.COM/|    1|
+-----------------+-----+
only showing top 3 rows



In [39]:
dic = dict(df_c.rdd.map(lambda x: (x['urls'], x['label'])).collect())

In [40]:
dic.get('http//0.ssbet138.cn/WWW.XPJ80886.COM/')

97

In [43]:
from pyspark.sql.functions import udf
def url_node(s):
    node = dic.get(s)
    return node

ur_udf = udf(url_node)

In [51]:
df_graph = df

In [52]:
#df1_graph = df1.map(url_node)
df_graph_new = df_graph.withColumn("source", ur_udf('source'))


In [53]:
df_graph_new = df_graph_new.withColumn("target", ur_udf('target'))

In [54]:
df_graph_new.take(5)

[Row(source='97', target='1029'),
 Row(source='97', target='3194'),
 Row(source='97', target='11110'),
 Row(source='97', target='2936'),
 Row(source='97', target='2114')]

In [55]:
df_graph_new.count()

27208

In [56]:
df_c.write.parquet("s3://aws-logs-780494741284-us-east-1/dataset/graph_node",mode="overwrite")

In [ ]:
df_graph_new.write.parquet("s3://aws-logs-780494741284-us-east-1/dataset/graph_edges",mode="overwrite")